In [1]:
import os
import numpy as np
import pandas as pd

In [5]:
print(wn.synsets('dog'))
print("\n")
print(wn.synsets('run'))
print("\n")
print(wn.synset('dog.n.01'))
print("\n")
print(wn.synset('run.v.01'))

[Synset('dog.n.01'), Synset('frump.n.01'), Synset('dog.n.03'), Synset('cad.n.01'), Synset('frank.n.02'), Synset('pawl.n.01'), Synset('andiron.n.01'), Synset('chase.v.01')]


[Synset('run.n.01'), Synset('test.n.05'), Synset('footrace.n.01'), Synset('streak.n.01'), Synset('run.n.05'), Synset('run.n.06'), Synset('run.n.07'), Synset('run.n.08'), Synset('run.n.09'), Synset('run.n.10'), Synset('rivulet.n.01'), Synset('political_campaign.n.01'), Synset('run.n.13'), Synset('discharge.n.06'), Synset('run.n.15'), Synset('run.n.16'), Synset('run.v.01'), Synset('scat.v.01'), Synset('run.v.03'), Synset('operate.v.01'), Synset('run.v.05'), Synset('run.v.06'), Synset('function.v.01'), Synset('range.v.01'), Synset('campaign.v.01'), Synset('play.v.18'), Synset('run.v.11'), Synset('tend.v.01'), Synset('run.v.13'), Synset('run.v.14'), Synset('run.v.15'), Synset('run.v.16'), Synset('prevail.v.03'), Synset('run.v.18'), Synset('run.v.19'), Synset('carry.v.15'), Synset('run.v.21'), Synset('guide.v.05'), Syns

In [3]:
files_text = []

for file in data.File.values :
    files_text.append (open (os.path.join ("dataset", file), 'r', encoding="cp437").read ())

In [4]:
print (len (files_text))

19


In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download ('punkt')
nltk.download ('stopwords')

def pre_process (_sentences) :
    # Get the Stopwords of English Language
    en_stops = set (stopwords.words ('english'))
    processed_sentences = []

    for sentence in _sentences :
        #print (sentence)
        sentence = re.sub (r'[^\w\s]','', sentence)
        lower_sen = sentence.lower ()
        token_sen = word_tokenize (lower_sen)

        #print (token_sen)

        new_sentence = []
        for word in token_sen:
            if word not in en_stops:
                new_sentence.append (word)
                #print (word)

        processed_sentences.append (new_sentence)
    return processed_sentences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shraeyas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shraeyas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
sentences = []
for file_text in files_text :
    sentences.append (pre_process ([file_text]))        
print (len(sentences))

19


In [7]:
from gensim.models.doc2vec import TaggedDocument
tagged_data = [TaggedDocument (words = _d[0], tags = [str (i)]) for i, _d in enumerate (sentences)]

In [8]:
print(tagged_data[0][0])
print(len(tagged_data))

['inheritance', 'basic', 'concept', 'objectoriented', 'programming', 'basic', 'idea', 'create', 'new', 'classes', 'add', 'extra', 'detail', 'existing', 'classes', 'done', 'allowing', 'new', 'classes', 'reuse', 'methods', 'variables', 'existing', 'classes', 'new', 'methods', 'classes', 'added', 'specialise', 'new', 'class', 'inheritance', 'models', 'γçiskindofγç', 'relationship', 'entities', 'objects', 'example', 'postgraduates', 'undergraduates', 'kinds', 'student', 'kind', 'relationship', 'visualised', 'tree', 'structure', 'γçÿstudentγçö', 'would', 'general', 'root', 'node', 'γçÿpostgraduateγçö', 'γçÿundergraduateγçö', 'would', 'specialised', 'extensions', 'γçÿstudentγçö', 'node', 'child', 'nodes', 'relationship', 'γçÿstudentγçö', 'would', 'known', 'superclass', 'parent', 'class', 'whereas', 'γçÿpostgraduateγçö', 'would', 'known', 'subclass', 'child', 'class', 'γçÿpostgraduateγçö', 'class', 'extends', 'γçÿstudentγçö', 'class', 'inheritance', 'occur', 'several', 'layers', 'visualised',

In [9]:
from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec (vector_size = 100, window = 19, min_count = 5, workers = 8, epochs = 1000, alpha = 0.03, min_alpha = 0.002)
model.build_vocab (tagged_data)
model.train (tagged_data, total_examples = model.corpus_count, epochs = model.epochs)

import os
model_path = os.path.join ("model", "Doc2Vec_Document.bin")
model.save(model_path)

In [10]:
test_sen = files_text [16]
processed = pre_process ([test_sen])

print ("V1_infer", processed[0] == tagged_data[0][0])

v1 = model.infer_vector (processed [0])
model.docvecs.most_similar (positive = [v1], topn = 5)

V1_infer False


<ipython-input-10-481c8ffb6752>:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  model.docvecs.most_similar (positive = [v1], topn = 5)


[('16', 0.9596492648124695),
 ('4', 0.8337626457214355),
 ('14', 0.6664733290672302),
 ('3', 0.5834048986434937),
 ('11', 0.5444937348365784)]